In [1]:
!uv pip install numpy pandas matplotlib tqdm datasets transformers evaluate

Audited 7 packages in 28ms


In [2]:
!uv pip install --pre torch torchvision torchaudio executorch --extra-index-url https://download.pytorch.org/whl/nightly/cpu

Audited 4 packages in 12ms


In [3]:
!uv pip install google google-genai

Audited 2 packages in 8ms


In [4]:
!uv pip install python-dotenv

Audited 1 package in 2ms


In [5]:
!uv pip install --upgrade accelerate optimum

Resolved 27 packages in 685ms                                        
Prepared 1 package in 1ms                                                
Uninstalled 1 package in 8ms
Installed 1 package in 7ms                                  
 - fsspec==2024.12.0
 + fsspec==2025.3.2


In [6]:
!python --version

Python 3.12.6


In [7]:
!uv pip install gptqmodel --no-build-isolation

Resolved 58 packages in 11ms                                         
Uninstalled 1 package in 2ms
Installed 1 package in 2ms                                  
 - fsspec==2025.3.2
 + fsspec==2024.12.0


In [8]:
!uv pip install tiktoken blobfile

Audited 2 packages in 3ms


In [9]:
!uv pip install huggingface_hub tiktoken torchtune sentencepiece tokenizers snakeviz lm_eval==0.4.5 blobfile

Audited 8 packages in 21ms


In [10]:
try:
    # If the import fails, this means there's nothing to remove
    import examples

    try:
        # If the import succeeds, this means it isn't using lm_eval's module
        import examples.models
    except:
        print(
            "Failed to import examples.models due to lm_eval conflict. Removing lm_eval examples module"
        )
        import shutil

        examples_path = examples.__path__[0]
        shutil.rmtree(examples_path)

except:
    pass

## Our code

In [11]:
from dotenv import load_dotenv
    
load_dotenv(dotenv_path='.env')

True

In [12]:
from src.turn_controller import TurnController

/Users/jayantdabas/Documents/GitHub/beyond-llm-surface-semantics/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
domain_path = {
    'politics': 'data/politics.json'
}

controller = TurnController(domain_path)

In [14]:
%%time
controller.run_scenario()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Running domain: politics with 10 scenarios:


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Our conversation history: 

[User]: 
Facts:
- The Coastal Protection Act requires a 30% reduction in industrial emissions by 2030.
- Senator Blackwood represents a state with significant coastal tourism.
- Senator Blackwood's campaign received substantial funding from traditional energy companies.
- Senator Blackwood has previously supported incremental environmental regulation.

Rules:
- R1: Senators typically prioritize economic interests of their major campaign donors.
- R2: Representatives of coastal regions generally support climate protection measures affecting their constituency.
- R3: Politicians who support incremental approaches typically oppose sweeping regulatory changes.

Preferences:
- P1: Regional economic interests take precedence over donor interests when directly conflicting.
- P2: Historical voting patterns predict future votes unless circumstances significantly change.

Answer the below question as "yes" or "no" based on the given facts, rules, and preferences. Outp

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Our conversation history: 

[User]: 
Facts:
- The Urban Mobility Initiative would fund public transit expansion in metropolitan areas.
- Mayor Rodriguez's city has significant traffic congestion problems.
- Previous transit projects in the city have experienced budget overruns.
- The initiative requires 30% matching funds from participating cities.

Rules:
- R1: Mayors typically support federal funding for local infrastructure priorities.
- R2: Historical project performance influences positions on similar new initiatives.
- R3: Local budget constraints affect support for matching fund requirements.

Preferences:
- P1: Infrastructure need outweighs historical project performance concerns.
- P2: Federal funding opportunities generally outweigh matching requirements.

Answer the below question as "yes" or "no" based on the given facts, rules, and preferences. Output should always start with "Answer: yes" or "Answer: no" and then explanation.

Example: output should always be in the forma

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Our conversation history: 

[User]: 
Facts:
- The Electoral Access Act would expand early voting periods to 14 days before elections.
- Commissioner Park oversees election administration in a county with historically long polling place lines.
- Commissioner Park previously implemented a weekend voting pilot program that increased turnout by 8%.
- The county has limited staff resources for extended election administration.

Rules:
- R1: Election administrators generally support measures addressing documented problems in their jurisdictions.
- R2: Successful pilot programs indicate support for similar expanded policies.
- R3: Resource constraints influence administrative positions on unfunded mandates.

Preferences:
- P1: Turnout improvement outweighs administrative burden concerns.
- P2: Evidence-based policy expansion outweighs resource limitation concerns.

Answer the below question as "yes" or "no" based on the given facts, rules, and preferences. Output should always start with "Ans

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Our conversation history: 

[User]: 
Facts:
- The Clean Energy Transition Act mandates 50% renewable energy by 2030.
- Governor Torres' state currently generates 22% of electricity from renewable sources.
- The state has significant natural gas production employing approximately 28,000 workers.
- Governor Torres campaigned on both economic development and environmental protection.

Rules:
- R1: Governors balance economic impacts against environmental goals in energy policy decisions.
- R2: States typically resist rapid transitions threatening major employment sectors.
- R3: Campaign themes establish general priority areas for gubernatorial policy decisions.

Preferences:
- P1: Protecting existing jobs takes precedence over creating potential future opportunities.
- P2: Environmental goals are moderated by economic impact assessments.

Answer the below question as "yes" or "no" based on the given facts, rules, and preferences. Output should always start with "Answer: yes" or "Answer: no

In [52]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
text = """answer: no

explanation: 
- the coastal protection act requires a 30% reduction in industrial emissions by 2030.
- senator blackwood represents a state wi """
scores = sid.polarity_scores(text)
print(scores)

{'neg': 0.104, 'neu': 0.896, 'pos': 0.0, 'compound': -0.296}


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jayantdabas/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [17]:
from src.builder.data_loader import DataLoader
from src.builder.prompt_generator import PromptGenerator

from src.model.inference import ModelInference

/Users/jayantdabas/Documents/GitHub/beyond-llm-surface-semantics/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
domain_path = {
    'politics': 'data/politics.json'
}

data_loader = DataLoader(domain_path)
prompt_generator = PromptGenerator()

model_inference = ModelInference()

# Load data
data = data_loader.get_data()

In [19]:
for domain, scenarios in data.items():
    # For each scenario
    for i, scenario in enumerate(scenarios):
        prompt = prompt_generator.generate_prompt(scenario)
        print(prompt)
        break

Facts:
- The Coastal Protection Act requires a 30% reduction in industrial emissions by 2030.
- Senator Blackwood represents a state with significant coastal tourism.
- Senator Blackwood's campaign received substantial funding from traditional energy companies.
- Senator Blackwood has previously supported incremental environmental regulation.

Rules:
- R1: Senators typically prioritize economic interests of their major campaign donors.
- R2: Representatives of coastal regions generally support climate protection measures affecting their constituency.
- R3: Politicians who support incremental approaches typically oppose sweeping regulatory changes.

Preferences:
- P1: Regional economic interests take precedence over donor interests when directly conflicting.
- P2: Historical voting patterns predict future votes unless circumstances significantly change.

Answer the below question as true or false based on the given facts, rules, and preferences. Output format should always start with "A

In [20]:
print(model_inference.chat(prompt))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


answer: true
explanation:
the coastal protection act requires a 30% reduction in industrial emissions by 2030.
senator blackwood represents a state with significant coastal tourism.
senator blackwood's campaign received substantial funding from traditional energy companies.
senator black


In [21]:
print(model_inference.chat(prompt))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


answer: true
explanation: senator blackwood represents a state with significant coastal tourism. senator blackwood's campaign received substantial funding from traditional energy companies. senator blackwood has previously supported incremental environmental regulation. therefore, we can conclude that senator blackwood will


In [22]:
print(model_inference.chat(prompt))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


answer: true

explanation: the coastal protection act requires a 30% reduction in industrial emissions by 2030. senator blackwood represents a state with significant coastal tourism. senator blackwood's campaign received substantial funding from traditional energy companies. senator black


In [11]:
print(model_inference.chat(prompt))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


answer: true

explanation: senator blackwood represents a state with significant coastal tourism. senator blackwood's campaign received substantial funding from traditional energy companies. senator blackwood has previously supported incremental environmental regulation. senator blackwood's campaign received substantial funding from traditional


In [12]:
print(model_inference.chat(prompt))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


answer: true


In [13]:
print(model_inference.chat(prompt))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


answer: false
explanation: 
the coastal protection act requires a 30% reduction in industrial emissions by 2030. senator blackwood represents a state with significant coastal tourism. senator blackwood's campaign received substantial funding from traditional energy companies. senator


## Gemini API

In [5]:
from dotenv import load_dotenv
    
load_dotenv(dotenv_path='.env')

True

In [9]:
import json
import numpy as np
import pandas

import base64
import os
import textwrap

# Tools from Hugging Face
from datasets import load_dataset
from transformers import pipeline

from google import genai
from google.genai import types

In [5]:
def load_data(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in: {file_path}")
        return None

In [ ]:
def generate_prompt(data):
    prompt = 'Answer the following question based on the given facts, rules, and preferences. Provide a true or false answer (binary cross-entropy) for each round, and make sure to take the information of initial setting and past rounds into consideration.\n\n'
    
    # Add facts to the prompt
    prompt += 'Facts:\n'
    for fact in data['facts']:
        prompt += f'- {fact}\n'

    # Add rules to the prompt
    prompt += '\nRules:\n'
    for rule in data['rules']:
        prompt += f"- {rule}\n"
    
    # Add preferences to the prompt
    prompt += '\nPreferences:\n'
    for preference in data['preferences']:
        prompt += f'- {preference}\n'
    
    # Add the question
    prompt += f'\nQuestion: {data['question']}\n'

    return prompt

In [7]:
data = load_data('data/politics.json')[0]
prompt = generate_prompt(data)
print(prompt)

Answer the following question based on the given facts, rules, and preferences. Provide a true or false answer (binary cross-entropy).

Facts:
- F1: The Coastal Protection Act requires a 30% reduction in industrial emissions by 2030.
- F2: Senator Blackwood represents a state with significant coastal tourism.
- F3: Senator Blackwood's campaign received substantial funding from traditional energy companies.
- F4: Senator Blackwood has previously supported incremental environmental regulation.

Rules:
- R1: Senators typically prioritize economic interests of their major campaign donors.
- R2: Representatives of coastal regions generally support climate protection measures affecting their constituency.
- R3: Politicians who support incremental approaches typically oppose sweeping regulatory changes.

Preferences:
- P1: Regional economic interests take precedence over donor interests when directly conflicting.
- P2: Historical voting patterns predict future votes unless circumstances signi

In [ ]:
def generate_response(prompt):
    client = genai.Client(
        api_key=os.environ.get('GEMINI_API_KEY'),
    )

    model = 'gemini-2.5-pro-exp-03-25'
    contents = [
        types.Content(
            role='user',
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_mime_type='text/plain',
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end='')

In [18]:
generate_response(prompt)

**Analysis:**

1.  **Pro-Vote Arguments (Supporting "True"):**
    *   F2 (Represents coastal state) + R2 (Coastal reps support climate protection) suggests a 'Yes'.
    *   P1 prioritizes regional interests (F2 - coastal tourism) over donor interests (F3 - energy companies), strengthening the R2 argument over the R1 argument.

2.  **Anti-Vote Arguments (Supporting "False"):**
    *   F3 (Energy funding) + R1 (Senators prioritize donors) suggests a 'No'. (However, P1 weakens this by prioritizing F2 over F3).
    *   F4 (Supports incremental regulation) + R3 (Incremental supporters oppose sweeping changes) suggests a 'No'. The 30% reduction in F1 sounds like a potentially "sweeping" change, not incremental.
    *   P2 (Historical patterns predict future votes) reinforces the F4+R3 argument. Blackwood's pattern is incrementalism, predicting opposition to non-incremental measures like the CPA.

3.  **Conflict Resolution:** The primary conflict is between the regional interest (F2+R2, stre